# EDA: Stanford Dogs Dataset

Exploratory Data Analysis для датасета Stanford Dogs (120 пород).

**Содержание:**
1. Загрузка датасета
2. Общая статистика (число классов, изображений)
3. Распределение классов (bar chart)
4. Разбивка train/val/test и баланс
5. Примеры изображений из разных классов
6. Распределение размеров изображений
7. Визуализация MixUp аугментации

In [ ]:
import sys
sys.path.insert(0, '..')

import json
import os
from pathlib import Path
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from torchvision.datasets import ImageFolder

sns.set_theme(style='whitegrid', palette='husl')
%matplotlib inline

DATA_ROOT = Path('../data')
SPLITS_DIR = DATA_ROOT / 'splits'

## 1. Загрузка датасета

In [ ]:
from src.dataset import find_images_dir

images_dir = find_images_dir(DATA_ROOT)
dataset = ImageFolder(root=str(images_dir))

print(f'Total images: {len(dataset)}')
print(f'Number of classes: {len(dataset.classes)}')
print(f'\nFirst 10 classes:')
for i, cls in enumerate(dataset.classes[:10]):
    # Убираем номер WordNet synset для читаемости
    breed = cls.split('-', 1)[-1] if '-' in cls else cls
    print(f'  {i}: {breed}')

## 2. Распределение классов

In [ ]:
# Подсчёт числа изображений на класс
targets = np.array(dataset.targets)
class_counts = Counter(targets)
counts = [class_counts[i] for i in range(len(dataset.classes))]

# Красивые имена пород
breed_names = [cls.split('-', 1)[-1].replace('_', ' ') if '-' in cls else cls 
               for cls in dataset.classes]

print(f'Min images per class: {min(counts)}')
print(f'Max images per class: {max(counts)}')
print(f'Mean images per class: {np.mean(counts):.1f}')
print(f'Std: {np.std(counts):.1f}')

In [ ]:
# Bar chart: распределение числа изображений по классам
fig, ax = plt.subplots(figsize=(20, 6))

sorted_indices = np.argsort(counts)[::-1]
sorted_counts = [counts[i] for i in sorted_indices]
sorted_names = [breed_names[i] for i in sorted_indices]

ax.bar(range(len(sorted_counts)), sorted_counts, color=sns.color_palette('viridis', len(sorted_counts)))
ax.set_xlabel('Class index (sorted by count)', fontsize=12)
ax.set_ylabel('Number of images', fontsize=12)
ax.set_title('Stanford Dogs: Distribution of Images per Class (120 breeds)', fontsize=14)
ax.axhline(y=np.mean(counts), color='red', linestyle='--', label=f'Mean = {np.mean(counts):.0f}')
ax.legend(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Гистограмма: сколько классов имеют N изображений
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(counts, bins=20, edgecolor='black', alpha=0.7)
ax.set_xlabel('Images per class', fontsize=12)
ax.set_ylabel('Number of classes', fontsize=12)
ax.set_title('Histogram: Class Size Distribution', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Разбивка train/val/test (70/15/15)

In [ ]:
from src.dataset import create_splits

splits = create_splits(dataset, SPLITS_DIR, ratios=(0.70, 0.15, 0.15), seed=42)

for name, indices in splits.items():
    split_targets = targets[indices]
    print(f'{name:>5}: {len(indices):>6} images | '
          f'classes represented: {len(np.unique(split_targets))}')

In [ ]:
# Проверяем баланс: число изображений на класс в каждом сплите
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

for ax, (name, indices) in zip(axes, splits.items()):
    split_targets = targets[indices]
    split_counts = Counter(split_targets)
    vals = [split_counts.get(i, 0) for i in range(len(dataset.classes))]
    
    ax.bar(range(len(vals)), sorted(vals, reverse=True), alpha=0.7)
    ax.set_title(f'{name} ({len(indices)} images)', fontsize=13)
    ax.set_xlabel('Class index')
    ax.set_ylabel('Count')
    ax.axhline(y=np.mean(vals), color='red', linestyle='--', alpha=0.7)

plt.suptitle('Class Distribution per Split', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 4. Примеры изображений

In [ ]:
# Показываем по 1 изображению из 16 случайных классов
rng = np.random.RandomState(42)
sample_classes = rng.choice(len(dataset.classes), size=16, replace=False)

fig, axes = plt.subplots(4, 4, figsize=(14, 14))

for ax, cls_idx in zip(axes.flat, sample_classes):
    # Берём первый индекс данного класса
    cls_indices = np.where(targets == cls_idx)[0]
    img_idx = rng.choice(cls_indices)
    img_path = dataset.imgs[img_idx][0]
    img = Image.open(img_path).convert('RGB')
    
    breed = breed_names[cls_idx]
    ax.imshow(img)
    ax.set_title(breed, fontsize=10)
    ax.axis('off')

plt.suptitle('Sample Images from Stanford Dogs', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

## 5. Распределение размеров изображений

In [ ]:
# Собираем размеры (width, height) для всех изображений
# Для скорости — берём подвыборку
sample_size = min(2000, len(dataset))
sample_indices = rng.choice(len(dataset), size=sample_size, replace=False)

widths, heights = [], []
for idx in sample_indices:
    path = dataset.imgs[idx][0]
    with Image.open(path) as img:
        w, h = img.size
        widths.append(w)
        heights.append(h)

print(f'Width  — min: {min(widths)}, max: {max(widths)}, mean: {np.mean(widths):.0f}')
print(f'Height — min: {min(heights)}, max: {max(heights)}, mean: {np.mean(heights):.0f}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(widths, bins=40, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_title('Image Width Distribution', fontsize=13)
axes[0].set_xlabel('Width (px)')
axes[0].set_ylabel('Count')

axes[1].hist(heights, bins=40, edgecolor='black', alpha=0.7, color='coral')
axes[1].set_title('Image Height Distribution', fontsize=13)
axes[1].set_xlabel('Height (px)')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
# Scatter: width vs height
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(widths, heights, alpha=0.3, s=10)
ax.set_xlabel('Width (px)', fontsize=12)
ax.set_ylabel('Height (px)', fontsize=12)
ax.set_title('Image Dimensions (Width vs Height)', fontsize=14)
ax.set_aspect('equal')
ax.plot([0, max(max(widths), max(heights))], [0, max(max(widths), max(heights))], 
        'r--', alpha=0.5, label='Square')
ax.legend()
plt.tight_layout()
plt.show()

## 6. Визуализация MixUp аугментации

In [ ]:
import torch
from src.dataset import get_dataloaders, get_mixup_fn, IMAGENET_MEAN, IMAGENET_STD

loaders = get_dataloaders(
    data_root=DATA_ROOT,
    splits_dir=SPLITS_DIR,
    image_size=224,
    batch_size=8,
    num_workers=0,
)

mixup_fn = get_mixup_fn(mixup_alpha=0.8, num_classes=120)

# Получаем один батч
images, labels = next(iter(loaders['train']))
print(f'Original batch: images {images.shape}, labels {labels.shape}')

# Применяем MixUp
mixed_images, mixed_labels = mixup_fn(images, labels)
print(f'Mixed batch: images {mixed_images.shape}, labels {mixed_labels.shape}')

In [ ]:
def denormalize(tensor, mean=IMAGENET_MEAN, std=IMAGENET_STD):
    """Денормализация для визуализации."""
    mean = torch.tensor(mean).view(3, 1, 1)
    std = torch.tensor(std).view(3, 1, 1)
    return (tensor * std + mean).clamp(0, 1)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i in range(4):
    # Original
    orig_img = denormalize(images[i]).permute(1, 2, 0).numpy()
    axes[0, i].imshow(orig_img)
    axes[0, i].set_title(f'Original (class {labels[i].item()})', fontsize=10)
    axes[0, i].axis('off')
    
    # Mixed
    mix_img = denormalize(mixed_images[i]).permute(1, 2, 0).numpy()
    # Top-2 классов из mixed labels
    top2 = mixed_labels[i].topk(2)
    title = f'Mixed ({top2.indices[0].item()}: {top2.values[0]:.2f}, {top2.indices[1].item()}: {top2.values[1]:.2f})'
    axes[1, i].imshow(mix_img)
    axes[1, i].set_title(title, fontsize=9)
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('Original', fontsize=13, rotation=0, labelpad=60)
axes[1, 0].set_ylabel('MixUp', fontsize=13, rotation=0, labelpad=60)

plt.suptitle('MixUp Augmentation (alpha=0.8)', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

## 7. Сводка

| Параметр | Значение |
|---|---|
| Датасет | Stanford Dogs |
| Число классов | 120 пород |
| Всего изображений | ~20,580 |
| Разбивка | 70% train / 15% val / 15% test |
| Аугментация | MixUp (alpha=0.8) |
| Размер входа | 224 x 224 |